缓存可以提升性能、缓解数据库压力，但是使用缓存也会导致数据不一致性的问题。一般我们是如何使用缓存呢？有三种经典的缓存使用模式：
* Cache-Aside Pattern
* Read-Through/Write-through
* Write-behind

#### Cache-Aside Pattern (旁路缓存模式)
读
* 读的时候，先读缓存，缓存命中的话，直接返回数据
* 缓存没有命中的话，就去读数据库，从数据库取出数据，放入缓存后，同时返回响应。

写 
更新的时候，先更新数据库，然后再删除缓存

#### Read-Through/Write-Through（读写穿透）
**Read-Through** 
* 从缓存读取数据，读到直接返回
* 如果读取不到的话，从数据库加载，写入缓存后，再返回响应。

Read-Through 实际只是在 Cache-Aside 之上进行了一层代码封装，它会让程序代码变得更简洁，同时也减少数据源上的负载。但读的思路一致


**Write-Through**
更新的时候，先更新数据库，然后再更新缓存 (cache aside 是删除缓存)


#### Write-behind （异步缓存写入） 
Write-behind 跟 Read-Through/Write-Through 有相似的地方，都是由 Cache Provider 来负责缓存和数据库的读写。它们又有个很大的不同：Read/Write-Through 是同步更新缓存和数据的，Write-Behind 则是只更新缓存，不直接更新数据库，通过批量异步的方式来更新数据库。   
这种方式下，缓存和数据库的一致性不强，对一致性要求高的系统要谨慎使用。但是它适合频繁写的场景，MySQL的InnoDB Buffer Pool机制就使用到这种模式. 


### Cach-Aside 操作缓存的问题
日常开发中，我们一般使用的就是Cache-Aside模式。Cache-Aside在写入请求的时候，为什么是删除缓存而不是更新缓存呢？     

#### 删除还是更新缓存    
首先, 多进程同时操作缓存的情况下, 更新缓存会带来数据不一致, 而采用删除缓存则不会.     
* 进程 A 先发起一个写操作，第一步先更新数据库   
* 进程 B 再发起一个写操作，第二步更新了数据库   
* 由于网络等原因，进程 B 先更新了缓存    
* 进程 A 更新缓存。    

其次,更新缓存相对于删除缓存，还有其它两点劣势：    
* 如果你写入的缓存值，是经过复杂计算才得到的话。更新缓存频率高的话，就浪费性能啦。    
* 在写数据库场景多，读数据场景少的情况下，数据很多时候还没被读取到，又被更新了，这也浪费了性能呢(实际上，写多的场景，用缓存也不是很划算的,哈哈)     


#### 双写的情况下，先操作数据库还是先操作缓存？

* 进程 A 发起一个写操作，第一步 del cache
* 此时进程 B 发起一个读操作，cache miss
* 进程 B 继续读 DB，读出来一个老数据
* 然后进程 B 把老数据设置入 cache
* 进程 A 写入 DB 最新的数据
酱紫就有问题啦，缓存和数据库的数据不一致了。缓存保存的是老数据，数据库保存的是新数据。因此，Cache-Aside缓存模式，选择了先操作数据库而不是先操作缓存。

个别小伙伴可能会问，先操作数据库再操作缓存，不一样也会导致数据不一致嘛？它俩又不是原子性操作的。这个是会的，但是这种方式，一般因为删除缓存失败等原因，才会导致脏数据，这个概率就很低。小伙伴们可以画下操作流程图，自己先分析下哈。接下来我们再来分析这种删除缓存失败的情况，如何保证一致性。


#### 数据库和缓存数据保持强一致，可以嘛？
实际上，没办法做到数据库与缓存绝对的一致性。   
* 加锁可以嘛？并发写期间加锁，任何读操作不写入缓存？      
* 缓存及数据库封装CAS乐观锁，更新缓存时通过lua脚本？    
* 分布式事务，3PC？TCC？     

其实，这是由 CAP 理论决定的。缓存系统适用的场景就是非强一致性的场景，它属于 CAP 中的 AP。个人觉得，追求绝对一致性的业务场景，不适合引入缓存。

#### 三种方案保证数据库与缓存的一致性      
1. 缓存延时双删     
  有些小伙伴可能会说，并不一定要先操作数据库呀，采用缓存延时双删策略，就可以保证数据的一致性啦。什么是延时双删呢？
    * 先删除缓存   
    * 再更新数据库    
    * 休眠一会（比如1秒），再次删除缓存。   
  
  这个休眠一会，一般多久呢？都是1秒？ `休眠时间 =  读业务逻辑数据的耗时 + 几百毫秒`。为了确保读请求结束，写请求可以删除读请求可能带来的缓存脏数据。   
  这种方案还算可以，只有休眠那一会（比如就那1秒），可能有脏数据，一般业务也会接受的。但是如果第二次删除缓存失败呢？缓存和数据库的数据还是可能不一致，对吧？给 Key 设置一个自然的 expire 过期时间，让它自动过期怎样？那业务要接受过期时间内，数据的不一致咯？还是有其他更佳方案呢？

2. 删除缓存重试机制    
 不管是延时双删还是 Cache-Aside 的先操作数据库再删除缓存，都可能会存在第二步的删除缓存失败，导致的数据不一致问题。可以使用这个方案优化：删除失败就多删除几次呀,保证删除缓存成功就可以了呀~ 所以可以引入删除缓存重试机制     
    * 写请求更新数据库
    * 缓存因为某些原因，删除失败
    * 把删除失败的 key 放到消息队列
    * 消费消息队列的消息，获取要删除的key
    * 重试删除缓存操作

3. 读取binlog异步删除缓存       
 重试删除缓存机制还可以吧，就是会造成好多业务代码入侵。其实，还可以这样优化：通过数据库的 binlog 来异步淘汰 key。   

 以mysql为例吧
    * 可以使用阿里的 canal 将 binlog 日志采集发送到 MQ 队列里面
    * 然后通过 ACK 机制确认处理这条更新消息，删除缓存，保证数据缓存一致性
























https://mp.weixin.qq.com/s/vYq9lRS8KeWOIqjCC01fqQ      差插图